## set DB

- Chroma DB 를 통해 VectorStore 를 구축한다.
- 벡터 저장소 생성 from_texts

In [ ]:
import torch
from load_embedding import load_embedding
from pdf_preprocessing import LegalText
from langchain_core.documents import Document
from langchain_chroma import Chroma
from glob import glob
import json

def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    elif torch.backends.mps.is_available():
        return torch.device("mps")
    return torch.device("cpu")

def load_pdf_documents(pdf_dir):
    """PDF 파일들을 읽어서 Document 객체 리스트로 변환합니다."""
    filenames = glob(f'{pdf_dir}/*.pdf')
    law_docs = []
    for filename in filenames:
        law_docs.extend(LegalText(filename).documents)
    return law_docs

def load_qna_data(file_path):
    """QnA JSON 파일을 읽어서 Document 객체 리스트로 변환합니다."""
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    documents = []
    for item in data:
        metadata = {
            **item.get('metadata', {}),
            'question': item['question']
        }
        
        doc = Document(
            page_content=item['answer'],
            metadata=metadata
        )
        documents.append(doc)
    
    return documents

def create_chroma_db():
    """Document 객체들로 새로운 Chroma DB를 생성합니다."""
    DB_PATH = "./chroma_db"
    PDF_DIR = "/workspace/LangEyE/App/src/crawling/files/laws"
    JSON_PATH = "/workspace/LangEyE/App/src/crawling/files/faq_results_20241211.json"

    device = get_device()
    
    embedding_model = load_embedding(device)
    
    pdf_documents = load_pdf_documents(PDF_DIR)

    qna_documents = load_qna_data(JSON_PATH)
    
    # 모든 문서 합치기
    all_documents = pdf_documents + qna_documents
    
    db = Chroma.from_documents(
        documents=all_documents,
        embedding=embedding_model,
        persist_directory=DB_PATH,
        collection_name="my_db"
    )
    db.persist()
    print(f"Successfully created Chroma DB with {len(all_documents)} total documents")
    return db

501


## QnA 데이터 셋 추가하기

In [ ]:
import json

def load_qna_data(file_path):
    """QnA JSON 파일을 읽어서 Document 객체 리스트로 변환합니다."""
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    documents = []
    for item in data:
        # 메타데이터 결합
        metadata = {
            **item.get('metadata', {}),
            'question': item['question']
        }
        
        # Document 객체 생성
        doc = Document(
            page_content=item['answer'],
            metadata=metadata
        )
        documents.append(doc)
    
    return documents

def add_to_chroma(documents, db_path, embedding_model):
    """Document 객체들을 Chroma DB에 추가합니다."""
    # 기존 DB에 연결
    db = Chroma(
        persist_directory=db_path,
        embedding_function=embedding_model,
        collection_name="my_db"
    )
    
    # 새로운 문서 추가
    db.add_documents(documents)
    db.persist()
    
    return db

def main():
    DB_PATH = "./chroma_db"
    JSON_PATH = "/Volumes/MINDB/24년/SW아카데미/LangEyE/App/src/crawling/files/faq_results_20241211.json"  # JSON 파일 경로
    DEVICE = "cuda"  # 또는 "cpu"
    
    # 임베딩 모델 로드
    embedding_model = load_embedding(DEVICE)
    
    # QnA 데이터 로드
    documents = load_qna_data(JSON_PATH)
    
    # Chroma DB에 추가
    db = add_to_chroma(documents, DB_PATH, embedding_model)
    print(f"Successfully added {len(documents)} documents to Chroma DB")

if __name__ == "__main__":
    main()

### 유사도 검색

`similarity_search` 메서드는 Chroma 데이터베이스에서 유사도 검색을 수행합니다. 이 메서드는 주어진 쿼리와 가장 유사한 문서들을 반환합니다.

**매개변수**

- `query` (str): 검색할 쿼리 텍스트
- `k` (int, 선택적): 반환할 결과의 수. 기본값은 4입니다.
- `filter` (Dict[str, str], 선택적): 메타데이터로 필터링. 기본값은 None입니다.

**참고**

- `k` 값을 조절하여 원하는 수의 결과를 얻을 수 있습니다.
- `filter` 매개변수를 사용하여 특정 메타데이터 조건에 맞는 문서만 검색할 수 있습니다.
- 이 메서드는 점수 정보 없이 문서만 반환합니다. 점수 정보도 필요한 경우 `similarity_search_with_score` 메서드를 직접 사용하세요.

**반환값**

- `List[Document]`: 쿼리 텍스트와 가장 유사한 문서들의 리스트

- query 에서 Pattern 을 찾아서 filter 를 적용한다.

In [ ]:
# filter 사용
db.similarity_search(
    "TF IDF 에 대하여 알려줘", filter={"source": "data/nlp-keywords.txt"}, k=2
)